In [1]:
import numpy as np 
import pandas as pd 
import math
import matplotlib.pyplot as plt 
# from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score,confusion_matrix

In [2]:
trainData = pd.read_csv("dataset/train_features.csv")
testData = pd.read_csv("dataset/test_features.csv")
data_cols = np.array(trainData.columns)
data_cols

array(['letter_slant', 'line_slant', 'letter_size', 'word_spacing',
       'personality'], dtype=object)

In [3]:
trainData.head()

,letter_slant,line_slant,letter_size,word_spacing,personality
0,2.0,-75.1,931.7,15.0,Agreeableness
1,-11.0,9.6,486.2,22.0,Agreeableness
2,18.0,-2.3,112.8,1.0,Agreeableness
3,6.0,-4.0,30.6,1.0,Agreeableness
4,-6.0,0.0,152.9,9.0,Agreeableness


In [4]:
testData.head()

,letter_slant,line_slant,letter_size,word_spacing,personality
0,0.0,1.0,421.2,14.0,Agreeableness
1,-3.0,1.8,94.8,22.0,Agreeableness
2,10.0,-1.9,1397.6,39.0,Agreeableness
3,0.0,2.8,84.2,71.0,Agreeableness
4,-2.0,-87.0,763.3,18.0,Agreeableness


In [5]:
k = int(input())
print("The value of k is:", k)

The value of k is: 7


In [6]:
trainData.isnull().sum()

letter_slant    0
line_slant      0
letter_size     0
word_spacing    0
personality     0
dtype: int64

In [7]:
testData.isnull().sum()

letter_slant    0
line_slant      0
letter_size     0
word_spacing    0
personality     0
dtype: int64

In [8]:
trainData = trainData.dropna(axis=0)
testData = testData.dropna(axis=0)

In [9]:
testData1 = testData.drop('personality', axis=1)
testData1.head()

,letter_slant,line_slant,letter_size,word_spacing
0,0.0,1.0,421.2,14.0
1,-3.0,1.8,94.8,22.0
2,10.0,-1.9,1397.6,39.0
3,0.0,2.8,84.2,71.0
4,-2.0,-87.0,763.3,18.0


In [10]:
def euclideanDistance(instance1,instance2,length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return np.sqrt(distance)

In [11]:
import operator
def getNeighbours(trainingSet,testSet,k):
    distance = []
    length = len(testSet)
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testSet,trainingSet.loc[x].values,length)
        # dist = euclideanDistance(testSet,trainingSet[x],length)
        distance.append([trainingSet.loc[x].values, dist])
    distance.sort(key=operator.itemgetter(1))
    neighbours = []
    for x in range(k):
        neighbours.append(distance[x][0])
    return neighbours

In [12]:
correct = 0
for key in range(len(testData)):
    Neighbours = getNeighbours(trainData, testData1.loc[key].values, k)
    targets = []
    for n in Neighbours:
        targets.append(n[-1])
    max_target = max(targets, key=targets.count)
    if testData['personality'][key] == max_target:
        correct += 1

accuracy = (correct / len(testData)) * 100
print(f'Accuracy: {accuracy}%')

Accuracy: 38.63636363636363%


In [13]:
Neighbours = getNeighbours(trainData,testData1.loc[5].values,k)
targets = []
for neighbour in Neighbours:
    targets.append(neighbour[-1])
Neighbours

[array([2.0, -2.1, 89.0, 37.0, 'Agreeableness'], dtype=object),
 array([10.0, -4.8, 99.6, 48.0, 'Openness'], dtype=object),
 array([8.0, 0.0, 82.9, 45.0, 'Openness'], dtype=object),
 array([4.0, -3.4, 82.9, 59.0, 'Conscientiousness'], dtype=object),
 array([7.0, -3.7, 88.4, 63.0, 'Conscientiousness'], dtype=object),
 array([-8.0, 6.7, 79.6, 44.0, 'Openness'], dtype=object),
 array([-11.0, 9.8, 103.5, 38.0, 'Openness'], dtype=object)]

In [14]:
targets

['Agreeableness',
 'Openness',
 'Openness',
 'Conscientiousness',
 'Conscientiousness',
 'Openness',
 'Openness']

In [15]:
max_target = max(targets, key=targets.count)
max_target

'Openness'